# ChatFlow Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_threads, run_dispatch_worker_thread
#from aiflows import flow_verse

#dependencies = [
#    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
#]
#flow_verse.sync_dependencies(dependencies)

# Connect to CoLink Server

In [3]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [4]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: Chatbot
flow_id: abb5db86-ed96-448d-917c-c796d390430e
message_ids: ['901df56d-dcba-4822-a656-eaa887c14702']
parallel_dispatch: False



/home/staverm/.pyenv/versions/3.10.6/lib/python3.10/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `Usage` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(



~~~ Dispatch task ~~~
flow_type: Chatbot
flow_id: ed3c4eeb-1f97-47cd-8536-a8b9ff9684fd
message_ids: ['7ed5825b-8530-48ef-8158-9452034a83ab']
parallel_dispatch: False



In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: Chatbot
flow_id: abb5db86-ed96-448d-917c-c796d390430e
message_ids: ['6f18b9dc-f905-4ee5-b6d2-0006b76be3fe']
parallel_dispatch: False


~~~ Dispatch task ~~~
flow_type: Chatbot
flow_id: ed3c4eeb-1f97-47cd-8536-a8b9ff9684fd
message_ids: ['bb3bdd9f-1fd3-45e5-a45b-d9f48b947dff']
parallel_dispatch: False



# Serve ChatFlow

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Chatbot"
)

Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Chatbot.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## Get two instances with different configurations

In [7]:
args_A = {
   "system_message_prompt_template": {
       "template": "You are a helpful chatbot that truthfully answers questions. Answer in the language English."
   }
}

chatbot_A = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_A,
)

Creating new flow instance with config:
 {
    "private_keys": [],
    "keys_to_ignore_for_hash_flow_config": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_flow_state": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_input_data": [],
    "clear_flow_namespace_on_run_end": true,
    "enable_cache": true,
    "name": "ChatAtomicFlow",
    "description": "Flow which uses as tool an LLM though an API",
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_name": "system",
    "user_name": "user",
    "assistant_name": "assistant",
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": [
            {
                "_target_": 

In [8]:
args_B = {
    "system_message_prompt_template": {
       "template": "You are a helpful chatbot that truthfully answers questions. Answer in the language Croatian."
   }
}

chatbot_B = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_B,
)

Creating new flow instance with config:
 {
    "private_keys": [],
    "keys_to_ignore_for_hash_flow_config": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_flow_state": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_input_data": [],
    "clear_flow_namespace_on_run_end": true,
    "enable_cache": true,
    "name": "ChatAtomicFlow",
    "description": "Flow which uses as tool an LLM though an API",
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_name": "system",
    "user_name": "user",
    "assistant_name": "assistant",
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": [
            {
                "_target_": 

## Conversation with chatbot A

In [9]:
input_message = FlowMessage(
    data={"id": 0, "query": "What is the capital of Switzerland?"},
)

response = chatbot_A.ask(input_message)
print(response.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

response = chatbot_A.ask(input_message)
print(response.get_data())

{'api_output': 'The capital of Switzerland is Bern.'}
{'api_output': 'Bern is located in the central part of Switzerland, on the Swiss Plateau, along the Aare River.'}


## Conversation with chatbot B

In [10]:
input_message = FlowMessage(
    data={"id": 0, "query": "What is the capital of Switzerland?"},
)

response = chatbot_B.ask(input_message)
print(response.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

response = chatbot_B.ask(input_message)
print(response.get_data())

{'api_output': 'Glavni grad Švicarske je Bern.'}
{'api_output': 'Bern se nalazi u središnjem dijelu Švicarske, na rijeci Aare.'}
